<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/Densenet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet121
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [ ]:
#######################

In [2]:
conv_base = DenseNet121(weights='imagenet', include_top = False, input_shape=(300,300,3))

29089792/29084464 [==============================] - 3s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [ ]:
# conv_base.summary()

In [4]:
import pickle
# X = pickle.load(open(f'pickle_files/X2669.pkl', 'rb'))
# y = pickle.load(open(f'pickle_files/y2669.pkl', 'rb'))
X = pickle.load(open("gdrive/My Drive/X2669.pkl", 'rb'))
y = pickle.load(open("gdrive/My Drive/y2669.pkl", 'rb'))



X = X.permute(0,2,3,1).numpy()
y = y.numpy()

dataset_size = X.shape[0]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
nos = X_train.shape[0]
nos

2135

In [7]:
((y==1).sum())/((y==0).sum()),((y_train==1).sum())/((y_train==0).sum()),((y_test==1).sum())/((y_test==0).sum())

(16.91275167785235, 15.811023622047244, 23.272727272727273)

In [8]:
y_test.shape, (y_test==1).sum(), (y_test==0).sum()

((534,), 512, 22)

In [9]:
# datagen = ImageDataGenerator(rescale=1./255, )
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [10]:
datagen.fit(X_train)

In [11]:
bs = 34
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [12]:
conv_base.predict(X_train[0].reshape(1,300,300,3)).shape


(1, 9, 9, 1024)

In [13]:
train_samples = np.zeros(shape=(nos,9,9,1024))
train_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    train_samples[i*bs:(i+1)*bs] = f
    train_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 1024)
(34, 9, 9, 102

In [14]:
del X_train    # deleting X_train to make space

In [15]:
train_samples = train_samples.reshape(nos,9*9*1024)
train_samples.shape, train_labels.shape

((2135, 82944), (2135,))

In [16]:
nostest = X_test.shape[0]
nostest

534

In [17]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [18]:
datagentest.fit(X_test)

In [19]:
numitest = NumpyArrayIterator(
    X_test, y_test, datagentest, batch_size=nostest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(nostest,9,9,1024))
test_labels = np.zeros(shape=(nostest))
for samples_batch,labels_batch in numitest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(534, 9, 9, 1024)


In [20]:
del X_test

In [21]:
test_samples = test_samples.reshape(nostest,9*9*1024)
test_samples.shape

(534, 82944)

In [22]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.Adam(lr = 1e-3),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
        


In [23]:
my_callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),

]

In [33]:
history = model.fit(train_samples, train_labels, epochs = 100, batch_size = 20,
                    validation_data = (test_samples, test_labels), callbacks = my_callbacks
)


Train on 2135 samples, validate on 534 samples
Epoch 1/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1217 - accuracy: 0.9447 - val_loss: 0.1409 - val_accuracy: 0.9588
Epoch 2/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1177 - accuracy: 0.9452 - val_loss: 0.1248 - val_accuracy: 0.9588
Epoch 3/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1069 - accuracy: 0.9447 - val_loss: 0.1555 - val_accuracy: 0.9569
Epoch 4/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1023 - accuracy: 0.9433 - val_loss: 0.1267 - val_accuracy: 0.9588
Epoch 5/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1038 - accuracy: 0.9443 - val_loss: 0.1572 - val_accuracy: 0.9607
Epoch 6/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.1002 - accuracy: 0.9438 - val_loss: 0.1266 - val_accuracy: 0.9588
Epoch 7/100
2135/2135 [==============================] - 4s 2ms/step - loss: 0.0840

In [ ]:
# run = 1      22:100

In [34]:
preds = model.predict(test_samples, batch_size = 20)
predclass = np.array([1.0 if i>0.5 else 0.0 for i in preds])
(predclass == test_labels).sum()/534.0

0.9588014981273408

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels,predclass)

array([[  0,  22],
       [  0, 512]])

In [36]:
confusion_matrix(test_labels,predclass)[0][0]/22, confusion_matrix(test_labels,predclass)[1][1]/512 

(0.0, 1.0)

In [ ]:
test_labels.shape

(534,)

Validation Data

In [ ]:
xval = np.load('gdrive/My Drive/Copy of xtrain.npy')
yval = np.load('gdrive/My Drive/Copy of ytrain.npy')


In [ ]:
yval.shape[0]

298

In [ ]:
numitval = NumpyArrayIterator(
    xval, yval, datagen, batch_size=yval.shape[0], shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


val_samples = np.zeros(shape=(yval.shape[0],9,9,512))
val_labels = np.zeros(shape=(yval.shape[0]))
for samples_batch,labels_batch in numitval:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    val_samples[::] = g
    val_labels[:] = labels_batch
    break

(298, 9, 9, 512)


In [ ]:
val_samples = val_samples.reshape(yval.shape[0],9*9*512)

In [ ]:
val_preds = model.predict(val_samples, batch_size = 20)

In [ ]:
val_predclass = np.array([1.0 if i>0.5 else 0.0 for i in val_preds])

In [ ]:

(val_predclass == val_labels).sum()/298.0

0.9228187919463087

In [ ]:
confusion_matrix(val_labels,val_predclass)

array([[126,  23],
       [  0, 149]])

In [ ]:
confusion_matrix(val_labels,val_predclass)[0][0]/149, confusion_matrix(val_labels,val_predclass)[1][1]/149 

(0.8456375838926175, 1.0)

In [ ]:
# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

Saved model to disk


In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/